<a href="https://colab.research.google.com/github/virendrasinh734/book_recommendation/blob/main/recommend_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-10-05 19:02:03--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   133MB/s    in 0.2s    

2023-10-05 19:02:04 (133 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [21]:
df_books = pd.read_csv(
    'BX-Books.csv',
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    'BX-Book-Ratings.csv',
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [22]:
print(df_books.shape)
print(df_ratings.shape)

(271379, 3)
(1149780, 3)


In [17]:
user_review_counts = df_ratings['user'].value_counts()

book_review_counts = df_ratings['isbn'].value_counts()

user_review_counts = df_ratings['user'].value_counts()
popular_users = user_review_counts[user_review_counts > 100].index
book_review_counts = df_ratings['isbn'].value_counts()
popular_books = book_review_counts[book_review_counts > 10].index

filtered_ratings = df_ratings[
    (df_ratings['user'].isin(popular_users)) &
    (df_ratings['isbn'].isin(popular_books))
]
final_data = filtered_ratings.merge(df_books, on='isbn')


In [18]:
final_data

,user,isbn,rating,title,author
0,276925,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
2,3363,002542730X,0.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
3,10030,002542730X,7.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
4,11676,002542730X,6.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
...,...,...,...,...,...
273476,252695,0140445285,0.0,Crime and Punishment (Penguin Classics),Fyodor Dostoyevsky
273477,264903,3789129402,10.0,"Ronja, RÃ?Â¤ubertochter. ( Ab 10 J.).",Astrid Lindgren
273478,268030,0141182679,0.0,On the Road (Penguin Modern Classics),Jack Kerouac
273479,276018,3498044761,0.0,Der TÃ?Â¤nzer.,Colum McCann


In [35]:
titles=final_data['title']
tr=titles.drop_duplicates()
tr

0         Politically Correct Bedtime Stories: Modern Ta...
102           Sushi for Beginners : A Novel (Keyes, Marian)
118               Wasted : A Memoir of Anorexia and Bulimia
130                               La casa de los espÃ­ritus
138                                     The Music of Chance
                                ...                        
273474    Bias : A CBS Insider Exposes How the Media Dis...
273477                Ronja, RÃ?Â¤ubertochter. ( Ab 10 J.).
273478                On the Road (Penguin Modern Classics)
273479                                       Der TÃ?Â¤nzer.
273480                                            Mondlaub.
Name: title, Length: 13629, dtype: object

In [36]:
table=final_data.pivot_table(index='title',columns='user',values='rating')
table.fillna(0,inplace=True)
table

user,254,507,882,1424,1435,1733,1903,2033,2110,2276,...,276463,276538,276680,276925,277427,277478,277639,278137,278188,278418
title,,,,,,,,,,,,,,,,,,,,,
Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Q-Space (Star Trek The Next Generation, Book 47)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Q-Zone (Star Trek The Next Generation, Book 48)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
!Yo!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
iI Paradiso Degli Orchi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
one hundred years of solitude,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
book_titles = table.index.tolist()

isbn_list = df_books[df_books['title'].isin(book_titles)]['isbn'].tolist()

print(isbn_list)

['0002005018', '0374157065', '0399135782', '1552041778', '1558746218', '0440234743', '0452264464', '0609804618', '1841721522', '0971880107', '0345402871', '0345417623', '0684823802', '0375759778', '0375406328', '0446310786', '0449005615', '0060168013', '055321215X', '0061099686', '0553582909', '0671888587', '0553582747', '0425182908', '042518630X', '0440223571', '0842342702', '0440225701', '0380715899', '0671623249', '0679810307', '0345260317', '042511774X', '0804106304', '1853262404', '1853260053', '0060938412', '0140067477', '0345465083', '0451625889', '1558531025', '0441783589', '0394895894', '0375410538', '0340767936', '0316769487', '8445071408', '0679429220', '0671867156', '0312252617', '0312261594', '0316748641', '0446677450', '0451166892', '067976397X', '0786868716', '0671864769', '0312953453', '0446608653', '0446612545', '0446612618', '0451208080', '0553584383', '0671027360', '0812575954', '0316735736', '0743439740', '0380619458', '0671042858', '0743249992', '0425184226', '0375

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time


output_directory = "filess"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

def fetch_book_description(isbn):
    url = f"https://openlibrary.org/isbn/{isbn}"

    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        description_element = soup.find('div', {'class': 'book-description'})

        if description_element:
            description = description_element.get_text(strip=True)
            return description

    return None

for isbn in isbn_list:
    description = fetch_book_description(isbn)
    if description:
        filename = f"{isbn}.txt"
        with open(os.path.join(output_directory, filename), "w", encoding="utf-8") as file:
            file.write(description)
            print(f"Saved description for ISBN {isbn} to {filename}")
    else:
        print(f"Description not found for ISBN {isbn}")
    time.sleep(2)

print("Done")


Saved description for ISBN 0002005018 to 0002005018.txt
Saved description for ISBN 0374157065 to 0374157065.txt
Saved description for ISBN 0399135782 to 0399135782.txt
Saved description for ISBN 1552041778 to 1552041778.txt
Saved description for ISBN 1558746218 to 1558746218.txt
Saved description for ISBN 0440234743 to 0440234743.txt
Saved description for ISBN 0452264464 to 0452264464.txt
Saved description for ISBN 0609804618 to 0609804618.txt
Saved description for ISBN 1841721522 to 1841721522.txt
Saved description for ISBN 0971880107 to 0971880107.txt
Saved description for ISBN 0345402871 to 0345402871.txt
Saved description for ISBN 0345417623 to 0345417623.txt
Saved description for ISBN 0684823802 to 0684823802.txt
Saved description for ISBN 0375759778 to 0375759778.txt
Saved description for ISBN 0375406328 to 0375406328.txt
Saved description for ISBN 0446310786 to 0446310786.txt
Saved description for ISBN 0449005615 to 0449005615.txt
Saved description for ISBN 0060168013 to 0060168

In [14]:
table.shape

(13629, 1818)

In [15]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13629 entries,  Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback)) to why I'm like this : True Stories
Columns: 1818 entries, 254 to 278418
dtypes: float32(1818)
memory usage: 94.6+ MB


In [25]:
import sqlite3


In [26]:

# Connect to the SQLite database (create a new database if it doesn't exist)
conn = sqlite3.connect('my_database.db')

# Create a new table 'titles_table' in the database with a column 'title'
tr.to_sql('table_name', conn, if_exists='replace', index=False, dtype={'title': 'TEXT'})

# Close the connection
conn.close()

In [27]:
def get_db_connection():
    conn = sqlite3.connect('my_database.db')
    return conn

def get_book_titles():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT title FROM table_name')
    titles = [row[0] for row in cursor.fetchall()]
    conn.close()
    return titles

# Assuming you have already connected to the SQLite database and assigned the connection to 'conn'

titles2 = get_book_titles()

In [ ]:
def get_table_schema(table_name):
    query = f"PRAGMA table_info({table_name})"
    cursor = conn.execute(query)
    schema = cursor.fetchall()

    # Convert the result to a dictionary for easy access
    table_schema = {}
    for column_info in schema:
        column_name, data_type, _, _, _, _ = column_info
        table_schema[column_name] = data_type

    return table_schema

# Get the schema of the newly created table
schema = get_table_schema('table_name')
print(schema)

# Close the connection
conn.close()

{0: '183', 1: '254', 2: '507', 3: '882', 4: '1424', 5: '1435', 6: '1733', 7: '1903', 8: '2033', 9: '2110', 10: '2276', 11: '2358', 12: '2442', 13: '2766', 14: '2891', 15: '2977', 16: '3363', 17: '3371', 18: '3757', 19: '3827', 20: '4017', 21: '4157', 22: '4385', 23: '5582', 24: '5903', 25: '6242', 26: '6251', 27: '6323', 28: '6543', 29: '6563', 30: '6575', 31: '6703', 32: '7105', 33: '7125', 34: '7158', 35: '7286', 36: '7346', 37: '7915', 38: '8066', 39: '8067', 40: '8245', 41: '8253', 42: '8284', 43: '8362', 44: '8454', 45: '8487', 46: '8681', 47: '8782', 48: '8890', 49: '8936', 50: '8971', 51: '9141', 52: '9417', 53: '9747', 54: '9856', 55: '9908', 56: '10030', 57: '10314', 58: '10354', 59: '10447', 60: '10560', 61: '10819', 62: '11120', 63: '11601', 64: '11629', 65: '11676', 66: '11724', 67: '11993', 68: '12538', 69: '12657', 70: '12824', 71: '12835', 72: '12982', 73: '13082', 74: '13093', 75: '13273', 76: '13540', 77: '13552', 78: '13582', 79: '13850', 80: '13935', 81: '13995', 82:

In [ ]:
from fuzzywuzzy import fuzz, process


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [28]:
nbrs = NearestNeighbors(n_neighbors=6, algorithm='auto', metric='cosine')
nbrs.fit(table)

# Compute the cosine similarity with indices
similarities, indices = nbrs.kneighbors(table)

# Transform the cosine similarity to get similarity between 0 and 1
closeness =1- similarities

print(closeness)
print(indices)

[[1.         0.76450956 0.71736455 0.68552434 0.64358056 0.63303417]
 [0.9999999  0.6154574  0.6154574  0.6154574  0.6154574  0.57143784]
 [1.         1.         1.         0.80582297 0.7432942  0.6902685 ]
 ...
 [0.99999994 0.5763904  0.5763904  0.5763904  0.5763904  0.5763904 ]
 [0.99999994 0.45145118 0.42161417 0.42062485 0.41401422 0.39517713]
 [1.         0.7624929  0.7624929  0.5674536  0.50210506 0.48074287]]
[[    0 13029  8497  6629  4070  6670]
 [    1   802  6666 11644 13065  2482]
 [ 7929 13521     2  4232  7263  9227]
 ...
 [13626 12895 11658  5890   511  8617]
 [13627  6430  8046  2541  5714  5235]
 [13628  8698  2975  2604  6253  5338]]


In [29]:
def get_recommends(book):
  ind=np.where(table.index==book)[0][0]
  temp=indices[ind]
  rc=[]
  for i in range(len(temp)):

    b=table.iloc[temp[i]].name
    # temp_df=df_books[df_books['title']==b]
    # temp_df.drop_duplicates('title')['title']
    s=closeness[ind][i]
    t2=[b]
    rc.append(b)
  recommended_books=[rc]


  return recommended_books

In [30]:
books = get_recommends("Anna Karenina")
print(books)

[['Anna Karenina', "Cleveland Amory's Compleat Cat: The Cat Who Came for Christmas : The Cat and the Curmudgeon, the Best Cat Ever", 'Cosmos', 'The Red Badge of Courage', 'Tunnel Vision (V.I. Warshawski Novels (Hardcover))', 'The Lost Continent: Travels in Small Town America']]


In [31]:
import pickle
pickle.dump(table,open('table.pkl','wb'))
pickle.dump(indices,open('indices.pkl','wb'))
pickle.dump(closeness,open('closeness.pkl','wb'))

In [33]:
booksdb = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author','y','p','img2','img'],
    usecols=['isbn', 'title', 'author','y','p','img2','img'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str','y':'str','p':'str','img2':'str','img':'str'})

In [34]:
pickle.dump(booksdb,open('booksdb.pkl','wb'))

In [32]:
pickle.dump(titles2,open('titles.pkl','wb'))

In [ ]:
booksdb

,isbn,title,author,y,p,img2,img
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271375,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271377,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...
